In [33]:
import pandas as pd
from keras.layers import Dense, Dropout, Activation, Input
from keras.models import Model 

In [34]:
base=pd.read_csv('games.csv')
base=base.drop('Other_Sales',axis=1)
base=base.drop('Global_Sales',axis=1)
base=base.drop('Developer',axis=1) 

In [35]:
base=base.dropna(axis=0)
base=base.loc[base['NA_Sales']>1]
base=base.loc[base['EU_Sales']>1]

In [36]:
base['Name'].value_counts()
nome_jogos=base.Name
base=base.drop('Name',axis=1)

In [37]:
previsores=base.iloc[:,[0,1,2,3,7,8,9,10,11]].values
venda_na=base.iloc[:,4].values
venda_eu=base.iloc[:,5].values
venda_jp=base.iloc[:,6].values

In [38]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
labelencoder_previsores = LabelEncoder()
previsores[:, 0] = labelencoder_previsores.fit_transform(previsores[:, 0])
previsores[:, 2] = labelencoder_previsores.fit_transform(previsores[:, 2])
previsores[:, 3] = labelencoder_previsores.fit_transform(previsores[:, 3])
previsores[:, 8] = labelencoder_previsores.fit_transform(previsores[:, 8])


In [39]:
categorical_columns = [0, 2, 3, 8]
onehotencoder = ct = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(), categorical_columns)],
    remainder='passthrough')
previsores = ct.fit_transform(previsores).toarray()

In [40]:
camada_entrada=Input(shape=(61,))
camada_oculta1=Dense(units=32,activation='sigmoid')(camada_entrada)
camada_oculta2=Dense(units=32,activation='sigmoid')(camada_oculta1)
camada_saida1=Dense(units=1,activation='linear')(camada_oculta2)
camada_saida2=Dense(units=1,activation='linear')(camada_oculta2)
camada_saida3=Dense(units=1,activation='linear')(camada_oculta2)

In [41]:
regressor=Model(inputs=camada_entrada,
                outputs=[camada_saida1,camada_saida2,camada_saida3])

In [42]:
regressor.compile(optimizer='adam', loss='mse')

In [43]:
regressor.fit(previsores,[venda_na,venda_eu,venda_jp], epochs=5000, batch_size=100)

Epoch 1/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 36.5987
Epoch 2/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 30.2402 
Epoch 3/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 34.3256 
Epoch 4/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 23.7858 
Epoch 5/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 33.0864 
Epoch 6/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 32.0423 
Epoch 7/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 23.1485 
Epoch 8/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 30.9959 
Epoch 9/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 24.9676 
Epoch 10/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 30.5509 
Epoch 11/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 20.9497
Epoch 12/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 19.4813 
Epoch 13/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 22.7210 
Epoch 14/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 19.5633 
Epoch 15/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 

In [ ]:
previsao_na,previsao_eu,previsao_jp = regressor.predict(previsores)